<a href="https://colab.research.google.com/github/Scyzentraz/modulajar/blob/main/Upscale.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os

if not os.path.exists("Real-ESRGAN"):
    !git clone https://github.com/xinntao/Real-ESRGAN.git

%cd Real-ESRGAN
!pip install basicsr facexlib gfpgan ffmpeg-python
!pip install -r requirements.txt
!python setup.py develop
!pip install tqdm

In [ ]:
file_path = "/usr/local/lib/python3.11/dist-packages/basicsr/data/degradations.py"

with open(file_path, 'r') as file:
    file_data = file.readlines()

# Hanya patch jika belum dipatch
patched = False
for i, line in enumerate(file_data):
    if 'from torchvision.transforms.functional_tensor import rgb_to_grayscale' in line:
        file_data[i] = 'from torchvision.transforms.functional import rgb_to_grayscale\n'
        patched = True
        break

if patched:
    with open(file_path, 'w') as file:
        file.writelines(file_data)
    print("✅ Bug torchvision berhasil dipatch.")
else:
    print("✅ Tidak perlu patch, sudah sesuai.")

In [ ]:
import os
from glob import glob

# Konfigurasi
source = "image"
input_dir = "/content/drive/MyDrive/upscale/input"
output_dir = "/content/drive/MyDrive/upscale/output"
model = "RealESRGAN_x4plus"
scale = 4
face_enhance = False

# Buat folder output jika belum ada
os.makedirs(output_dir, exist_ok=True)

# Ambil semua file jpg/jpeg
image_paths = sorted(
    glob(os.path.join(input_dir, "*.jpg")) +
    glob(os.path.join(input_dir, "*.jpeg")) +
    glob(os.path.join(input_dir, "*.png"))
)
print(f"Jumlah gambar ditemukan: {len(image_paths)}")

In [ ]:
from tqdm import tqdm

code = "inference_realesrgan.py"

# Jalankan batch dengan progress bar
for input_path in tqdm(image_paths, desc="🚀 Memproses gambar"):
    filename = os.path.basename(input_path)
    output_path = os.path.join(output_dir, filename)

    if face_enhance:
        !python {code} -i "{input_path}" -n {model} -o "{output_path}" -s {scale} --face_enhance
    else:
        !python {code} -i "{input_path}" -n {model} -o "{output_path}" -s {scale}

In [ ]:
from IPython.display import Image, display

for input_path in image_paths[:3]:
    filename = os.path.basename(input_path)
    output_path = os.path.join(output_dir, filename)
    print(f"\n📎 Before: {filename}")
    display(Image(filename=input_path, width=360))
    print(f"✨ After: {filename}")
    display(Image(filename=output_path, width=360))